In [1]:
# Here is the imports
from tensorflow import keras
import numpy as np
#import matplotlib.pylot as plt
import tensorflow as tf
import segmentation_models as sm
sm.set_framework('tf.keras')
import os
from matplotlib import pyplot as plt
import cv2
# segmentation_models could also use `tf.keras` if you do not have Keras installed
# or you could switch to other framework using `sm.set_framework('tf.keras')`
#aman amana nerelere geldik

2021-08-12 22:20:07.814282: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Segmentation Models: using `tf.keras` framework.


<h1>CONSTANTS</h1>

In [ ]:
# Dataset Constants
DATASET_PATH = ".\\dataset\\"
TRAIN_DIR = "train"
VAL_DIR = "validation"
TEST_DIR = "test"

DATA_DIR = "data"
LABEL_DIR = "label"

# Model Constants
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 2
# unlabelled 0, iskemik 1, hemorajik 2
MASK_CLASSES = ['background', 'iskemik', 'kanama']
# format image 512,512,3 mask 512,512,class_num each 512,512 mask involves true and false for each class
CLASSES = ['iskemik', 'kanama']
LR = 0.0001
EPOCHS = 40
MODEL_SAVE_PATH = "./models"

In [ ]:
# Variables
x_train_dir = os.path.join(DATASET_PATH, TRAIN_DIR, DATA_DIR)
y_train_dir = os.path.join(DATASET_PATH, TRAIN_DIR, LABEL_DIR)

x_val_dir = os.path.join(DATASET_PATH, VAL_DIR, DATA_DIR)
y_val_dir = os.path.join(DATASET_PATH, VAL_DIR, LABEL_DIR)

x_test_dir = os.path.join(DATASET_PATH, TEST_DIR, DATA_DIR)
y_test_dir = os.path.join(DATASET_PATH, TEST_DIR, LABEL_DIR)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint('./best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]

# Dataloader and utility functions 

In [ ]:
# helper function for data visualization
def visualize(**images):
    """Plot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

def visualize_dataset(img, mask, classes):
    kwarg = {'image': img}
    for i in range(len(classes+1)):
        kwarg.update({classes[i] : mask[..., i].squeeze()})
    visualize(**kwarg)

    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x
    

# classes for data loading and preprocessing
class Dataset:
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        self.CLASSES = MASK_CLASSES
        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        # extract certain classes from mask (e.g. cars)
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))
        self.on_epoch_end()

    def __getitem__(self, i):    
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        # newer version of tf/keras want batch to be in tuple rather than list
        return tuple(batch)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)   

# Sanity Check Dataset

In [ ]:
dataset = Dataset(x_train_dir, y_train_dir, classes=CLASSES)

image, mask = dataset[5] # get some sample

tf.data.Dataset.from_tensor_slices() # make tf.dataset and save it as tfRecord
visualize_dataset(image, mask, CLASSES)

In [ ]:
def preprocessing() -> tf.data.Dataset:
    preprocessing_fn = sm.get_preprocessing(BACKBONE)
    
    pass

In [ ]:
def model_factory() -> keras.Model:
    # define network parameters
    n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
    activation = 'sigmoid' if n_classes == 1 else 'softmax'

    #create model
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

    # define optomizer
    optim = keras.optimizers.Adam(LR)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    # set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
    # TODO redefine class weights
    dice_loss = sm.losses.DiceLoss(class_weights=np.array([1, 2, 0.5])) 
    focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
    total_loss = dice_loss + (1 * focal_loss)

    # actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
    # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

    # compile keras model with defined optimozer, loss and metrics
    model.compile(optim, total_loss, metrics)
    return model

In [ ]:
def train_model(data_generators, model, callbacks):
    train_dataloader = data_generators[0]
    valid_dataloader = data_generators[1]
    # train model
    history = model.fit(
        train_dataloader, 
        steps_per_epoch=len(train_dataloader), 
        epochs=EPOCHS, 
        callbacks=callbacks, 
        validation_data=valid_dataloader, 
        validation_steps=len(valid_dataloader),
    )
    # TODO make this after eval for writing accuracy in the model name
    save_path = os.path.join(MODEL_SAVE_PATH, f"{}acc_{}.{}")
    model.save(save_path)

In [ ]:
def evaluate_model(eval_generator, model):
    pass

In [ ]:
def plot_history(train_history):
    # Plot training & validation iou_score values
    plt.figure(figsize=(30, 5))
    plt.subplot(121)
    plt.plot(train_history.history['iou_score'])
    plt.plot(train_history.history['val_iou_score'])
    plt.title('Model iou_score')
    plt.ylabel('iou_score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(train_history.history['loss'])
    plt.plot(train_history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# TODO Main code comes here
generators = preprocessing()
model = model_factory()


In [4]:
model_double_unet = sm.custom.Double_Unet(backbone_name="vgg19", encoder_weights=None, input_shape=(512,512,3), classes=6)
# for the class problem solution is simple make softmax like sigmoid, if background class has the heighest probility count that pixel as 0
# if another class is higher count that as 1. think about another way to make 2nd model consider whole image. Also in the last layer
# you can add class probility


Starting up Double Unet initialization...
Processing settings...
Settings initialized. Starting to build model.
ASPP-1 initialized...
Decoder-1 initialized...
Output-1 initialized...
output is KerasTensor(type_spec=TensorSpec(shape=(None, 512, 512, 6), dtype=tf.float32, name=None), name='activation_3/Softmax:0', description="created by layer 'activation_3'")
x is KerasTensor(type_spec=TensorSpec(shape=(None, 512, 512, 6), dtype=tf.float32, name=None), name='tf.math.multiply_2/Mul:0', description="created by layer 'tf.math.multiply_2'")
Encoder-2 initialized...
ASPP-2 initialized...
Decoder-2 initialized...
Output-2 initialized...
Model initialization complete.


In [ ]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
)

In [ ]:
model_double_unet.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_double_unet.fit(X_train, y_train, batch_size = 32, epochs = 50, callbacks=[tensorboard_callback])